This notebook contains the exploratory analysis for creating a linear regression model to predict ratings (cleanliness and review score) from other predictor variables contained within the `listings_df.csv` file from the kaggle Seattle AirBnB dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
from mpl_toolkits.basemap import Basemap

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline

In [2]:
pd.__version__

'0.25.0'

In [3]:
listings_df = pd.read_csv('../data/listings.csv')

In [4]:
listings_df.shape

(3818, 92)

In [5]:
print(listings_df.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [6]:
#https://datascience.stackexchange.com/questions/39137/how-can-i-check-the-correlation-between-features-and-target-variable
# df_dummy[df_dummy.columns[0:].corr()]['review_scores_cleanliness'][:]

Looking at these columns, I'm going to guess which ones might possibly be related to the cleanliness rating. I'll pare it down to just a small selection:

In [7]:
cols_of_interest = ['room_type','amenities', 'price', 'cleaning_fee', 'security_deposit', 'review_scores_cleanliness']

cols_of_interest = ['neighbourhood_group_cleansed', 'latitude', 'longitude',
                    'property_type', 'room_type', 'bed_type', 'amenities', 
                    'price', 'security_deposit', 'cleaning_fee', 'review_scores_cleanliness']

In [8]:
listings_df = listings_df[cols_of_interest]
listings_df.head()

,room_type,amenities,price,cleaning_fee,security_deposit,review_scores_cleanliness
0,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,NaN,NaN,10.0
1,Entire home/apt,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,$100.00,10.0
2,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,"$1,000.00",10.0
3,Entire home/apt,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,NaN,NaN,NaN
4,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,$700.00,9.0


In [9]:
#find categorical columns and display them
listings_cat_cols = listings_df.select_dtypes(include='object')
listings_cat_cols.columns

Index(['room_type', 'amenities', 'price', 'cleaning_fee', 'security_deposit'], dtype='object')

In [10]:
room_type_cleanliness = listings_df.copy().groupby(['room_type']).agg(['mean','count'])
room_type_cleanliness[[('review_scores_cleanliness','mean'),('review_scores_cleanliness','count')]].sort_values(by=('review_scores_cleanliness','mean'),ascending=False)

review_scores_cleanliness      
                                     mean count
room_type                                      
Entire home/apt                  9.587376  2123
Private room                     9.542105   950
Shared room                      8.989130    92

```
bed_type_cleanliness = listings_df.copy().groupby(['bed_type']).agg(['mean','count'])
bed_type_cleanliness[[('review_scores_cleanliness','mean'),('review_scores_cleanliness','count')]].sort_values(by=('review_scores_cleanliness','mean'),ascending=False)
```

Note that some of these categorical columns are prices, which we should probably convert to values. We can do this using `str.replace` to get rid of the $ and , and casting to a new type.

We can convert these to floats by removing the $ and , and casting the type to float

In [11]:
price_cols = ['price','cleaning_fee','security_deposit']
for col in price_cols:
    listings_df[col] = listings_df[col].str.replace('[\$,]', '', regex=True).astype(float)

In [12]:
listings_df[['price','cleaning_fee','security_deposit']].head()

,price,cleaning_fee,security_deposit
0,85.0,NaN,NaN
1,150.0,40.0,100.0
2,975.0,300.0,1000.0
3,100.0,NaN,NaN
4,450.0,125.0,700.0


In [13]:
listings_df[['price','cleaning_fee','security_deposit']].count()

price               3818
cleaning_fee        2788
security_deposit    1866
dtype: int64

Lastly, before we create dummies for the categorical variables, let's see how many different options there are for each of them: 

In [14]:
for col in listings_df.select_dtypes(include='object'):
    print(col + ': ' + str(len(listings_df[col].unique())))

room_type: 3
amenities: 3284


We need to do something about the `amenities` column, since this probably has some influence on the rating, but right now there are way too many values in this to just throw it into the model as-is. Let's find a list of all the unique amenities, not a unique set of all the lists. We can do this with the newer pandas `explode` method, but first we need to clean this column so that each entry is a list, and strip out the superfluous characters such as `{`  and `}`

In [15]:
# Clean amenities lambda function
clean_amenities = lambda col: col.strip('{}').split(',')
# Apply the function to the 'amenities' column
listings_df['amenities_list'] = listings_df['amenities'].apply(clean_amenities)

In [16]:
listings_df['amenities_list']

0       [TV, "Cable TV", Internet, "Wireless Internet"...
1       [TV, Internet, "Wireless Internet", Kitchen, "...
2       [TV, "Cable TV", Internet, "Wireless Internet"...
3       [Internet, "Wireless Internet", Kitchen, "Indo...
4       [TV, "Cable TV", Internet, "Wireless Internet"...
                              ...                        
3813    [TV, "Cable TV", Internet, "Wireless Internet"...
3814    [TV, "Cable TV", Internet, "Wireless Internet"...
3815    ["Cable TV", "Wireless Internet", Kitchen, "Fr...
3816    [TV, "Wireless Internet", Kitchen, "Free Parki...
3817    [TV, "Cable TV", Internet, "Wireless Internet"...
Name: amenities_list, Length: 3818, dtype: object

Using tips from this stackoverflow post, we can extract the unique values using `explode`:
https://stackoverflow.com/questions/51813266/get-unique-values-from-pandas-series-of-lists

In [17]:
set(listings_df['amenities_list'].explode())

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Washer / Dryer"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [18]:
listings_df['amenities_list'].explode().value_counts()

"Wireless Internet"             3667
Heating                         3627
Kitchen                         3423
"Smoke Detector"                3281
Essentials                      3237
Dryer                           2997
Washer                          2992
Internet                        2811
Shampoo                         2670
TV                              2574
"Carbon Monoxide Detector"      2485
"Fire Extinguisher"             2196
"Free Parking on Premises"      2167
"Family/Kid Friendly"           1963
"First Aid Kit"                 1680
"Cable TV"                      1446
"Indoor Fireplace"               886
"Pets live on this property"     883
Hangers                          846
"Elevator in Building"           785
"Hair Dryer"                     774
"Laptop Friendly Workspace"      745
Iron                             742
"Safety Card"                    727
"Air Conditioning"               677
"24-Hour Check-in"               616
"Buzzer/Wireless Intercom"       538
D

In [19]:
len(listings_df['amenities_list'].explode().value_counts())

42

There are still 42 unique values for the amenities, reading through the list let's make some separate columns for the amenities which we think might affect cleanliness, like pets, fireplace, or smoking.

In [20]:
listings_df['pet_friendly'] = listings_df['amenities'].str.contains('|'.join(['Pet','pet','Cat','Dog']))
listings_df['pet_friendly'] = listings_df['pet_friendly'].map({True: 'Yes', False: 'No'})

In [21]:
listings_df[['pet_friendly','amenities']].head()

,pet_friendly,amenities
0,No,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A..."
1,No,"{TV,Internet,""Wireless Internet"",Kitchen,""Free..."
2,Yes,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A..."
3,No,"{Internet,""Wireless Internet"",Kitchen,""Indoor ..."
4,No,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki..."


In [22]:
listings_df['smoker_friendly'] = listings_df['amenities'].str.contains('Smoking Allowed')
listings_df['smoker_friendly'] = listings_df['smoker_friendly'].map({True: 'Yes', False: 'No'})

In [23]:
listings_df['has_fireplace'] = listings_df['amenities'].str.contains('Fireplace')
listings_df['has_fireplace'] = listings_df['has_fireplace'].map({True: 'Yes', False: 'No'})

In [24]:
pet_cleanliness = listings_df.copy().groupby(['pet_friendly']).agg(['mean','count'])
pet_cleanliness[[('review_scores_cleanliness','mean'),('review_scores_cleanliness','count')]].sort_values(by=('review_scores_cleanliness','mean'),ascending=False)

review_scores_cleanliness      
                                  mean count
pet_friendly                                
Yes                           9.570858  1002
No                            9.549699  2163

In [25]:
smoker_cleanliness = listings_df.copy().groupby(['smoker_friendly']).agg(['mean','count'])
smoker_cleanliness[[('review_scores_cleanliness','mean'),('review_scores_cleanliness','count')]].sort_values(by=('review_scores_cleanliness','mean'),ascending=False)

review_scores_cleanliness      
                                     mean count
smoker_friendly                                
No                               9.559574  3097
Yes                              9.411765    68

In [26]:
listings_df['cat_friendly'] = listings_df['amenities'].str.contains('Cat')
listings_df['cat_friendly'] = listings_df['cat_friendly'].map({True: 'Yes', False: 'No'})

cat_cleanliness = listings_df.copy().groupby(['cat_friendly']).agg(['mean','count'])
cat_cleanliness[[('review_scores_cleanliness','mean'),('review_scores_cleanliness','count')]].sort_values(by=('review_scores_cleanliness','mean'),ascending=False)

review_scores_cleanliness      
                                  mean count
cat_friendly                                
Yes                           9.654321   324
No                            9.545231  2841

In [27]:
listings_df['dog_friendly'] = listings_df['amenities'].str.contains('Dog')
listings_df['dog_friendly'] = listings_df['dog_friendly'].map({True: 'Yes', False: 'No'})

dog_cleanliness = listings_df.copy().groupby(['dog_friendly']).agg(['mean','count'])
dog_cleanliness[[('review_scores_cleanliness','mean'),('review_scores_cleanliness','count')]].sort_values(by=('review_scores_cleanliness','mean'),ascending=False)

review_scores_cleanliness      
                                  mean count
dog_friendly                                
Yes                           9.593458   428
No                            9.550603  2737

You'd need to find a way to include Cat and not Dog or vice versa to do a real comparison, likely most have both unless allergies are involved. 

In [28]:
for col in listings_df.select_dtypes(include='object'):
    try:
        print(col + ': ' + str(len(listings_df[col].unique())))
    except:
        print(col + ': n/a')

room_type: 3
amenities: 3284
amenities_list: n/a
pet_friendly: 2
smoker_friendly: 2
has_fireplace: 2
cat_friendly: 2
dog_friendly: 2


In [29]:
listings_df = listings_df.drop(['amenities','amenities_list'],axis=1)

This is a more reasonable number of predictors to work with. Still probably too many, but at least we have a decent start.

In [30]:
listings_df.shape

(3818, 10)

In [31]:
listings_df.columns

Index(['room_type', 'price', 'cleaning_fee', 'security_deposit',
       'review_scores_cleanliness', 'pet_friendly', 'smoker_friendly',
       'has_fireplace', 'cat_friendly', 'dog_friendly'],
      dtype='object')

In [32]:
list(listings_df.select_dtypes(include='object').columns)

['room_type',
 'pet_friendly',
 'smoker_friendly',
 'has_fireplace',
 'cat_friendly',
 'dog_friendly']

Let's read in the function we used to create dummy variables for the categorical variables:

In [33]:
#Create a list of the column names of the categorical variables
cat_cols_lst = list(listings_df.select_dtypes(include='object').columns)

def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - boolean holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

In [34]:
df_dummy = create_dummy_df(listings_df, cat_cols_lst,dummy_na=False)
list(df_dummy.columns)

['price',
 'cleaning_fee',
 'security_deposit',
 'review_scores_cleanliness',
 'room_type_Private room',
 'room_type_Shared room',
 'pet_friendly_Yes',
 'smoker_friendly_Yes',
 'has_fireplace_Yes',
 'cat_friendly_Yes',
 'dog_friendly_Yes']

```
def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    
    Your function should:
    1. Drop the rows with missing response values
    2. Drop columns with NaN for all the values
    3. Use create_dummy_df to dummy categorical columns
    4. Fill the mean of the column for any missing values 
    5. Split your data into an X matrix and a response vector y
    6. Create training and test sets of data
    7. Instantiate a LinearRegression model with normalized data
    8. Fit your model to the training data
    9. Predict the response for the training data and the test data
    10. Obtain an rsquared value for both the training and test data
    '''
    #Drop the rows with missing response values
    df  = df.dropna(subset=[response_col], axis=0)

    #Drop columns with all NaN values
    df = df.dropna(how='all', axis=1)

    #Dummy categorical variables
    df = create_dummy_df(df, cat_cols, dummy_na)

    # Mean function
    fill_mean = lambda col: col.fillna(col.mean())
    # Fill the mean
    df = df.apply(fill_mean, axis=0)

    #Split into explanatory and response variables
    X = df.drop(response_col, axis=1)
    y = df[response_col]

    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)

    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X_train, y_train) #Fit

    #Predict using your model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test
```

In [35]:
def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    
    Your function should:
    1. Drop the rows with missing response values
    2. Drop columns with NaN for all the values
    3. Use create_dummy_df to dummy categorical columns
    4. Fill the mean of the column for any missing values 
    5. Split your data into an X matrix and a response vector y
    6. Create training and test sets of data
    7. Instantiate a LinearRegression model with normalized data
    8. Fit your model to the training data
    9. Predict the response for the training data and the test data
    10. Obtain an rsquared value for both the training and test data
    '''
    #Drop the rows with missing response values
    df  = df.dropna(subset=[response_col], axis=0)

    #Drop columns with all NaN values
    df = df.dropna(how='all', axis=1)

    df.dropna(subset=['room_type'],axis=0)
    
    #Dummy categorical variables
    df = create_dummy_df(df, cat_cols, dummy_na)

#     # Mean function
#     fill_mean = lambda col: col.fillna(col.mean())
#     # Fill the mean
#     df = df.apply(fill_mean, axis=0)

    #Split into explanatory and response variables
    X = df.drop(response_col, axis=1)
    y = df[response_col]

    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)

    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X_train, y_train) #Fit

    #Predict using your model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

In [36]:
#Test function with the dataset
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(listings_df[['room_type','review_scores_cleanliness']],'review_scores_cleanliness', cat_cols_lst, dummy_na=False)

In [37]:
print("The r-squared train score and test score were %.2f and %.2f" % (train_score,test_score))

The r-squared train score and test score were 0.01 and 0.02
